# Deep Convolutional Generative Adversarial Networks (DCGAN)

A model skeleton for Generative Adversarial Networks (GAN)s. It was first introduced in the paper titled *[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661)* by Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio.

> A generative adversarial network (GAN) is a class of machine learning frameworks designed by Ian Goodfellow and his colleagues in June 2014. Two neural networks contest with each other in a game (in the form of a zero-sum game, where one agent's gain is another agent's loss).

> Given a training set, this technique learns to generate new data with the same statistics as the training set. For example, a GAN trained on photographs can generate new photographs that look at least superficially authentic to human observers, having many realistic characteristics. Though originally proposed as a form of generative model for unsupervised learning, GANs have also proved useful for semi-supervised learning, fully supervised learning, and reinforcement learning.

> The core idea of a GAN is based on the "indirect" training through the discriminator, another neural network that can tell how "realistic" the input seems, which itself is also being updated dynamically. This means that the generator is not trained to minimize the distance to a specific image, but rather to fool the discriminator. This enables the model to learn in an unsupervised manner.

> GANs are similar to mimicry in evolutionary biology, with an evolutionary arms race between both networks.

## usage

In [1]:
import deeply
import deeply.datasets as dd
from deeply.transformers.scaler import image_scaler

mnist, info = dd.load("mnist", shuffle_files = True, as_supervised = True, with_info = True)
image_shape = info.features["image"].shape
n_classes   = info.features["label"].num_classes

gan = deeply.hub("dcgan", input_shape = image_shape, n_classes = n_classes,
                 decoder_batch_norm = True, encoder_dropout_rate = 0.3,
                 encoder_layer_growth_rate = 2, scaler = image_scaler,
                 init_decoder_units = 256, decoder_layer_growth_rate = 0.5, kernel_size = 5,
                 decoder_strides  = [1, 2], final_activation = "tanh")

Metal device set to: Apple M1


2022-10-11 19:09:06.997431: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-11 19:09:06.997531: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## example

In [2]:
import tensorflow as tf
from deeply.plots import imgplot as imgplt, mplt

from tqdm.auto import tqdm, trange

def mapper(image, label):
    return image_scaler.fit_transform(image), tf.one_hot(label, n_classes)

config = dict(batch_size = 256,
              epochs = 20)
batch_size = config["batch_size"]

data   = mnist["train"].map(mapper)
batch  = data.batch(batch_size)

In [ ]:
from deeply.model.gan import save_img_samples

samples = gan.get_random_latent_vector(shape = (batch_size, None))

for e in trange(config["epochs"]):
    for b in tqdm(batch):
        history = gan.fit(b, batch_size = batch_size, checkpoint_path = "gan.h5d5")
    save_img_samples(gan, samples = samples, to_file = "cgan.png")

  0%|          | 0/20 [00:00<?, ?it/s]

2022-10-11 19:09:10.437545: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  0%|          | 0/235 [00:00<?, ?it/s]

2022-10-11 19:09:11.301982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/achillesrasquinha/dev/deeply/src/deeply/plots.py:113: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = mplt.subplots(n_plots, 1, sharex = True)
